In [36]:
import mlrun

In [37]:
project = mlrun.get_or_create_project("drift-detection", "./", user_project=False)

> 2024-02-06 15:06:15,361 [info] Project loaded successfully: {'project_name': 'drift-detection'}


In [48]:
project.build_config(
    image="aipc_image",
    set_as_default=True,
    with_mlrun=False,
    base_image="mlrun/mlrun",
    requirements=["evidently"]
)
#project.build_image()
#This is the preferred approach for production workloads.
#project_set_source(source='git:// repo of aipc card')
project.export("text.yaml")

In [49]:
!ls

01-mlrun-basics.ipynb	      example.ipynb
02-model-training.ipynb       index.md
03-model-serving.ipynb	      project.yaml
04-pipeline.ipynb	      __pycache__
05-model-monitoring.ipynb     run_drift_detection.py
06-add-mlops-to-code.ipynb    src
07-batch-infer.ipynb	      text.yaml
colab			      utils.py
drift_detection_evidently.py


In [50]:
!ls /home/albana/ai_product_card_templates/document_classification_model/fasttext

ai_data_validation	 cards_metadata  ipzs		project-config.yaml
aipc_plain_execution.py  fastText-0.9.2  Makefile	__pycache__
ai_task_implementation	 input-folder	 output-folder	Readme.md


In [51]:
!cp /home/albana/ai_product_card_templates/document_classification_model/fasttext/ai_task_implementation/pipeline/drift_detection_evidently.py .

# Create function for drift detection

In [52]:
%%writefile run_drift_detection.py

import mlrun
import utils
import importlib

@mlrun.handler(outputs=["drift_result"])
def run_drift_detection():
    validation_card_metadata = utils.load_validation_card_specifications()

    specification = validation_card_metadata["specification"]
    implementation_file = specification["implementation"]
    package = specification["package"] 
    handler = specification["handler"]

    reference_data = utils.read_reference_dataset(validation_card_metadata["reference_data"])
    current_data = reference_data[reference_data['year'] > 2000]

    implementation_module = importlib.import_module(implementation_file)
    implementation = getattr(implementation_module, handler)
    result = implementation(reference_data, current_data)
    return result

Overwriting run_drift_detection.py


In [63]:
project.set_function(
    "run_drift_detection.py",
    name="run-drift-detection",
    kind="job",
    image="mlrun/mlrun",
    handler="run_drift_detection",
    requirements=["evidently"]
)
#project.build_function(
#   "run-drift-detection",
#   base_image="mlrun/mlrun",
#   commands= [
#        "pip install evidently",    
#   ]
#)


> 2024-02-06 15:56:27,995 [warning] Failed to add git metadata, ignore if path is not part of a git repo.: {'path': './', 'error': '/home/jovyan/tutorial'}


In [65]:
!pip install evidently
project.run_function("run-drift-detection", local=True, auto_build=True)
project.save()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 4.5 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 374.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 550.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 315.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.4/773.4 kB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 78.5 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0
  Attempting uninstall: uvicorn
    Found existing installation: uvico

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
drift-detection,...68c7db6f,0,Feb 06 15:57:12,completed,run-drift-detection-run-drift-detection,kind=owner=jovyanhost=ca2e72edab07,,,drift_result=False,


> 2024-02-06 15:57:22,194 [info] Run execution finished: {'status': 'completed', 'name': 'run-drift-detection-run-drift-detection'}


## Export AI Product project configurations (objects, functions, workflows, artifacts)

In [22]:
project_file_path = "/home/albana/ai_product_card_templates/document_classification_model/fasttext/project-config.yaml"
project.export(project_file_path)

In [21]:
serving_fn = mlrun.new_function(
    "serving",
    image="python:3.9",
    kind="serving",
    requirements=["mlrun[complete]", "scikit-learn~=1.3.0"],
)

function: auto-trainer
Automatic train, evaluate and predict functions for the ML frameworks - Scikit-Learn, XGBoost and LightGBM.
default handler: train
entry points:
  train: Training a model with the given dataset.

example::

    import mlrun
    project = mlrun.get_or_create_project("my-project")
    project.set_function("hub://auto_trainer", "train")
    trainer_run = project.run(
        name="train",
        handler="train",
        inputs={"dataset": "./path/to/dataset.csv"},
        params={
            "model_class": "sklearn.linear_model.LogisticRegression",
            "label_columns": "label",
            "drop_columns": "id",
            "model_name": "my-model",
            "tag": "v1.0.0",
            "sample_set": "./path/to/sample_set.csv",
            "test_set": "./path/to/test_set.csv",
            "CLASS_solver": "liblinear",
        },
    )
    context(MLClientCtx)  - MLRun context
    dataset(DataItem)  - The dataset to train the model on. Can be either a URI 